# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [43]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [41]:
# load messages dataset
messages = pd.read_csv('../data/disaster_messages.csv')
print(messages.head())
print()
print(messages.shape)

   id                                            message  \
0   2  Weather update - a cold front from Cuba that c...   
1   7            Is the Hurricane over or is it not over   
2   8                    Looking for someone but no name   
3   9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4  12  says: west side of Haiti, rest of the country ...   

                                            original   genre  
0  Un front froid se retrouve sur Cuba ce matin. ...  direct  
1                 Cyclone nan fini osinon li pa fini  direct  
2  Patnm, di Maryani relem pou li banm nouvel li ...  direct  
3  UN reports Leogane 80-90 destroyed. Only Hospi...  direct  
4  facade ouest d Haiti et le reste du pays aujou...  direct  

(26248, 4)


In [3]:
# load categories dataset
categories = pd.read_csv('../data/disaster_categories.csv')
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [5]:
# merge datasets

# use inner join as this removes entries that have eighter missing messages  
# OR missing categories. Neighter of these would be of any use later on.
df = pd.merge(messages, categories, how='inner', on=['id'])
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [11]:
transform_cat_df = categories['categories'].str.split(';', expand=True)
print(transform_cat_df[0].head())
print(transform_cat_df[0].str.replace('-\d*', '').head())
print(transform_cat_df[0].head())

0    related-1
1    related-1
2    related-1
3    related-1
4    related-1
Name: 0, dtype: object
0    related
1    related
2    related
3    related
4    related
Name: 0, dtype: object
0    related-1
1    related-1
2    related-1
3    related-1
4    related-1
Name: 0, dtype: object


In [12]:
labels = []
messy_columns = []

for i in transform_cat_df.columns:
    # check if each column contains only one label (<label>-<set|unset>)
#    work_df[i].str.split('-')
    col_i_series = transform_cat_df[i].str.replace('-\d*', '')
    transform_cat_df[i] = transform_cat_df[i].str.replace('.*-', '').astype(int)
    eq = col_i_series.value_counts() == transform_cat_df.shape[0]
    if( eq.all() ):
        label = col_i_series[0]
        labels.append(label)
        print("[*] {:2} {}".format(len(labels), label))
    else:
        messy_columns.append(i)
        print("[>] Column {} has mixed labels".format(i))
        
print()
print("[*] Found {} labels".format(len(labels)))
if( len(labels) == len(transform_cat_df.columns) ):
    print("[*] All columns have a unique label")
else:
    print("[x] Some columns are ambigous! Take a closer look at columns {}".format(messy_columns))
    
len_of_uniques = len(set(labels))
if(len_of_uniques != len(labels)):
    print("[x] Found {} duplicate labels".format(len(labels)-len_of_uniques))
    

transform_cat_df.columns = labels

[*]  1 related
[*]  2 request
[*]  3 offer
[*]  4 aid_related
[*]  5 medical_help
[*]  6 medical_products
[*]  7 search_and_rescue
[*]  8 security
[*]  9 military
[*] 10 child_alone
[*] 11 water
[*] 12 food
[*] 13 shelter
[*] 14 clothing
[*] 15 money
[*] 16 missing_people
[*] 17 refugees
[*] 18 death
[*] 19 other_aid
[*] 20 infrastructure_related
[*] 21 transport
[*] 22 buildings
[*] 23 electricity
[*] 24 tools
[*] 25 hospitals
[*] 26 shops
[*] 27 aid_centers
[*] 28 other_infrastructure
[*] 29 weather_related
[*] 30 floods
[*] 31 storm
[*] 32 fire
[*] 33 earthquake
[*] 34 cold
[*] 35 other_weather
[*] 36 direct_report

[*] Found 36 labels
[*] All columns have a unique label


In [14]:
transform_cat_df['id'] = categories['id']
categories = transform_cat_df
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,id
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,1,0,0,1,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,7
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12


In [15]:
# create a dataframe of the 36 individual category columns
df = pd.merge(messages, categories, how='inner', on=['id'])
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# select the first row of the categories dataframe
row = 

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
category_colnames = 
print(category_colnames)

In [ ]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.head()

### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [ ]:
for column in categories:
    # set each value to be the last character of the string
    categories[column] = 
    
    # convert column from string to numeric
    categories[column] = 
categories.head()

### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [ ]:
# drop the original categories column from `df`


df.head()

In [ ]:
# concatenate the original dataframe with the new `categories` dataframe
df = 
df.head()

### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [ ]:
df.isna().sum()

In [26]:
# check number of duplicates
def print_duplicates(df, column:str):
    unique_dates, count = np.unique(df[column].tolist(), return_counts=True)

    found_none_unique = False
    for i in range(len(unique_dates)):
        if count[i] >= 2:
            print("{:2}x {:6}".format(count[i], unique_dates[i]))
            found_none_unique = True

    if( not found_none_unique ): 
        print("No duplicates in column {}".format(column))
        
print_duplicates(df, 'id')
print_duplicates(df, 'message')

 4x    202
 4x    804
 4x    862
 4x   1652
 4x   2446
 4x   3217
 4x   3250
 4x   3882
 4x   4412
 4x   4956
 4x   5153
 4x   5265
 4x   5573
 4x   5643
 4x   5776
 4x   6327
 4x   6393
 4x   6492
 4x   6515
 4x   6573
 4x   6687
 4x   7747
 4x   7892
 4x   7945
 4x   8190
 4x   9131
 4x  10286
 4x  11503
 4x  12416
 4x  12420
 4x  12589
 4x  13059
 4x  13914
 4x  14073
 4x  14135
 4x  14246
 4x  14592
 4x  15169
 4x  15576
 4x  15760
 4x  15938
 4x  16245
 4x  17079
 4x  17385
 4x  17553
 4x  17569
 4x  17919
 4x  18313
 4x  18925
 4x  19003
 4x  19142
 4x  19687
 4x  21338
 4x  22059
 4x  22858
 4x  23002
 4x  23229
 4x  24247
 4x  24347
 9x  24779
 4x  25512
 4x  27296
 4x  27768
 4x  28462
 4x  28687
 4x  29022
 4x  29119
 4x    MAOZA          U  REP '
 4x #NAME?
 4x (Delmas 33 Charboniere infomatyon s'il vous plait.) 
 4x * German Agro Action (GAA) reported that in the 6 weeks up to 15 July it had completed the distribution of ECHO food to almost 14,000 beneficiaries in the Tavil

In [47]:
# remove missing messages
df = df[df['message'] != '#NAME?']
df.loc[df['related'] > 1, 'related'] = 1
print()

# drop duplicates
df.drop_duplicates(subset=['id', 'message'], inplace=True)

In [48]:
# check number of duplicates

print(df.isna().sum())
print_duplicates(df, 'id')
print_duplicates(df, 'message')
print(df.shape)

id                            0
message                       0
original                  16023
genre                         0
related                       0
request                       0
offer                         0
aid_related                   0
medical_help                  0
medical_products              0
search_and_rescue             0
security                      0
military                      0
child_alone                   0
water                         0
food                          0
shelter                       0
clothing                      0
money                         0
missing_people                0
refugees                      0
death                         0
other_aid                     0
infrastructure_related        0
transport                     0
buildings                     0
electricity                   0
tools                         0
hospitals                     0
shops                         0
aid_centers                   0
other_in

MemoryError: Unable to allocate 1.05 GiB for an array with shape (26176,) and data type <U10818

In [52]:
df['related'].value_counts()

2

### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [51]:
engine = create_engine('sqlite:///DisasterMessageClassification.sqlite')
df.to_sql('Messages', engine, index=False)

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.